# Práctica CNN - Clasificación de imágenes

En esta práctica vais a implementar un modelo de clasificación de principio a fin. Esto es, cargado y preparación de los datos, implementación del modelo, entrenamiento del mismo y testeo.

Para ello, vamos a emplear una base de datos de imágenes de 5 clases de flores diferentes. La objetivo es desarrollar un modelo de clasificación que sea capaz de diferenciar las distintas clases.

Para descargar las imágenes emplea el siguiente código:

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model

data_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

Si vemos la estructura de carpetas de esta base de datos vemos que tenemos una única carpeta dentro de la cual se encuentran todas las imágenes organizadas en carpetas en función de su clase. Con este vamos a crear 2 datasets, el correspondiente a entrenamiento y el correspondiente a validación, haciendo uso de un **tamaño de imagen de 224x224** y un **tamaño de lote de 32**.

In [ ]:
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
RANDOM_SEED = 2055

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,
    validation_split=0.2,
    subset="training",
    seed=RANDOM_SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Con esto implementa los siguientes pasos:

1. Generación de los datasets de entrenamiento y validación (haciendo uso del código anterior). Una vez generado el dataset, visualiza algunas imágenes para garantizar el cargado correcto de las mismas.

In [ ]:
train_dataset.class_names

In [ ]:
sample_images = train_dataset.take(1)

for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_dataset.class_names[labels[i]])
        plt.axis("off")

2. Normalización de las imágenes en un rango entre 0 y 1.

In [ ]:
def scaling_normalization(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label


train_dataset = train_dataset.map(scaling_normalization)
validation_dataset = validation_dataset.map(scaling_normalization)

3. Implementación de una arquitectura basada en redes neuronales convolucionales para la resolución de un problema de clasificación multiclase.

In [ ]:
input_layer = layers.Input(shape=[IMG_SIZE[0], IMG_SIZE[1], 3])

layer_conv1_1 = layers.Conv2D(filters=2, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")(
    input_layer)
layer_conv1_2 = layers.Conv2D(filters=2, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")(
    layer_conv1_1)
layer_pool1 = layers.MaxPool2D(pool_size=(2, 2))(layer_conv1_2)

layer_conv3_1 = layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")(
    layer_pool1)
layer_conv3_2 = layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")(
    layer_conv3_1)
layer_flatten = layers.Flatten()(layer_conv3_2)

dense_hidden1 = layers.Dense(32, activation='relu')(layer_flatten)

dense_output = layers.Dense(5, activation='softmax')(dense_hidden1)

model = Model(inputs=[input_layer], outputs=[dense_output])

4. Compilación del modelo haciendo uso de una función de pérdidas "sparse_categorical_crossentropy".

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

5. Entrenamiento del modelo durante 15 épocas.

In [95]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./flower_classifier.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

history_model_base = model.fit(train_dataset, epochs=15, callbacks=[model_checkpoint_callback],
                               validation_data=validation_dataset, verbose=1)

Epoch 1/15
90/92 [============================>.] - ETA: 0s - loss: 0.3203 - accuracy: 0.8993
Epoch 1: val_accuracy improved from -inf to 0.45640, saving model to .\flower_classifier.h5
92/92 [==============================] - 2s 23ms/step - loss: 0.3183 - accuracy: 0.9005 - val_loss: 1.9720 - val_accuracy: 0.4564
Epoch 2/15
89/92 [============================>.] - ETA: 0s - loss: 0.2512 - accuracy: 0.9291
Epoch 2: val_accuracy did not improve from 0.45640
92/92 [==============================] - 2s 23ms/step - loss: 0.2501 - accuracy: 0.9288 - val_loss: 2.1669 - val_accuracy: 0.4387
Epoch 3/15
91/92 [============================>.] - ETA: 0s - loss: 0.1930 - accuracy: 0.9499
Epoch 3: val_accuracy did not improve from 0.45640
92/92 [==============================] - 2s 23ms/step - loss: 0.1923 - accuracy: 0.9499 - val_loss: 2.4012 - val_accuracy: 0.4564
Epoch 4/15
90/92 [============================>.] - ETA: 0s - loss: 0.1491 - accuracy: 0.9681
Epoch 4: val_accuracy did not improve fr

6. Visualización de las curvas de aprendizaje de entrenamiento y validación.

In [ ]:
def plot_losses(history):
    plt.plot(history.history['loss'], label="Entrenamiento")
    plt.plot(history.history['val_loss'], label="Validación")
    plt.ylabel('Pérdidas')
    plt.xlabel('Época')
    plt.legend(loc="upper right")
    plt.title('Pérdidas durante el entrenamiento')


plot_losses(history_model_base)